In [40]:
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, train_test_split, cross_validate, \
                                    LeaveOneGroupOut, cross_val_score
from sklearn.metrics import roc_auc_score

In [189]:
# 5 regions
emb1 = pd.read_csv('/volatile/jl277509/Runs/02_STS_babies/Output/5_regions_5000_gamma/18-46-39_3/troiani_custom_embeddings/custom_cross_val_embeddings.csv')
# 5 regions acc grad
#emb1 = pd.read_csv('/volatile/jl277509/Runs/02_STS_babies/Output/5_regions_5000_acc_grads/12-31-55_18/troiani_custom_embeddings/custom_cross_val_embeddings.csv')
# FIP + orbital
#emb1 = pd.read_csv('/volatile/jl277509/Runs/02_STS_babies/Output/multiregion_single_encoder_test/13-45-25_52/troiani_custom_embeddings/custom_cross_val_embeddings.csv')
#emb1 = pd.read_csv('/volatile/jl277509/Runs/02_STS_babies/Program/Output/ORBITAL_12-layer_k7/20-56-02_1/troiani_custom_embeddings/custom_cross_val_embeddings.csv')
emb2 = pd.read_csv('/volatile/jl277509/Runs/02_STS_babies/Program/Output/ORBITAL_12-layer_k7/20-56-02_0/troiani_custom_embeddings/custom_cross_val_embeddings.csv')

In [199]:
# merge embeddings from two different trainings
emb2.columns = ['ID'] + [f'dim{k+257}' for k in range(256)]
embs = pd.merge(emb1,emb2)

In [200]:
## case with single embedding
#embs = emb1.copy()

In [201]:
# merge embeddings and labels
labels = pd.read_csv('/neurospin/dico/data/deep_folding/current/datasets/hcp/hcp_OFC_labels_from_0.csv', usecols=['Subject', 'Left_OFC'])
labels.columns = ['ID', 'Left_OFC']

embs = pd.merge(embs, labels)

In [202]:
X = embs.loc[:, embs.columns != 'ID']
X = embs.loc[:, embs.columns != 'Left_OFC']
Y = embs['Left_OFC']

In [203]:
# get the custom cv
splits_basedir = '/neurospin/dico/data/deep_folding/current/datasets/orbital_patterns/Troiani/train_val_split_'
subjects = embs['ID'].tolist()
root_dir = '/'.join(splits_basedir.split('/')[:-1])
basedir = splits_basedir.split('/')[-1]
splits_dirs = [os.path.join(root_dir,f) for f in os.listdir(root_dir) if basedir in f and '.csv' in f]
splits_subs = [pd.read_csv(file, header=None) for file in splits_dirs]
labels = np.concatenate([[i] * len(K) for i, K in enumerate(splits_subs)])
splits_subs_and_labels = pd.concat(splits_subs)
splits_subs_and_labels.columns=['ID']
splits_subs_and_labels['labels'] = labels
subs_embeddings = pd.DataFrame({'ID': subjects, 'X': list(X.values), 'Y': Y})
df = subs_embeddings.merge(splits_subs_and_labels, on='ID')
groups, X, Y = df['labels'], np.vstack(df['X'].values), df['Y']
logo = LeaveOneGroupOut()
cv = logo.get_n_splits(groups=groups)

In [204]:
model = LogisticRegression(max_iter=1000,
                           random_state=0)

In [205]:
labels_proba = cross_val_predict(model, X, Y, cv=cv, method='predict_proba')

/volatile/jl277509/Runs/02_STS_babies/Program/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/volatile/jl277509/Runs/02_STS_babies/Program/2023_jlaval_STSbabies/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for a

In [206]:
roc_aucs = roc_auc_score(Y, labels_proba, multi_class='ovr', average=None)

In [212]:
print(roc_aucs)
print(np.sum([roc_auc*np.sum(Y==k) for k, roc_auc in enumerate(roc_aucs)]) / len(Y))

[0.75952881 0.82782773 0.76549086 0.84150757]
0.7835335660505998


In [198]:
## values for single model alone !
print(roc_aucs)
print(np.sum([roc_auc*np.sum(Y==k) for k, roc_auc in enumerate(roc_aucs)]) / len(Y))

[0.76558748 0.82557379 0.75976745 0.82044108]
0.7835014674688842
